<h1>Elliptic Cruve Diffie-Hellman </h1>
in this notebook i'll try to explain the Diffie-Hellman's key exchange protocol Elleptic Cuvre version.
<h2>Elliptic Curve over finite fields </h2>
in the DH protocol we work on the integers group $ Z_{n} $, but in the ECDH we will work with EC over finite fields and we note $ E(\mathbb{F}_{p}) $,
in a clear term in an EC over finite field we are interested on the EC's points that belong to that fields 
(e.g if $ \mathbb{F} = \mathbb{Q}$ we are interested only on  the rational points of curve E.)
and for the DH we will chose $\mathbb{Z}_{p}$ as a field where p is a (256 bits) prime

In [56]:
from Crypto.Util.number import getPrime,inverse
from Crypto.Util.Padding import pad, unpad
import matplotlib.pyplot as plt
from Crypto.Cipher import AES
from random import randint
import numpy as np
import hashlib
import os




In [1]:
"p = getPrime(256)"
p = 85347454784018416893325519594186155831927915289184679356907899837050668265637

In [36]:
class EllipticCurve(object):
     def __init__(self, a, b, p):
        self.a = a 
        self.b = b
        self.p = p
        self.delta = -16 * (4 * a**3 + 27 * b**2) % self.p
        if not self.is_smooth():
            print(f"the curve {self} is not smooth!!")
        
     def is_smooth(self):
        return self.delta != 0
    
    
     def is_in_the_curve(self, x, y):
        return pow(y, 2,self.p)  ==  ((pow(x, 3,self.p) + self.a * x + self.b) % self.p)
    
    
     def __str__(self):
        return (f"y^2 = x^3 + {self.a}x + {self.b} mod  {self.p}")
     def visualize_cruve(self):
          y, x = np.ogrid[-4:4:100j, -4:4:100j]
          a,b=self.a,self.b
          plt.contour(x.ravel(), y.ravel(),pow(y,2) - pow(x,3) - a * x + b, [0])
          plt.show()

In [51]:
class Point(object): 
    
    def __init__(self, EC, x, y):
            self.EC = EC
            self.p = self.EC.p
            self.x = x % self.p
            self.y = y % self.p
        
            
            if (not EC.is_in_the_curve(self.x, self.y)) and ( (x!=0) and (y!=0)):
                print(f"The point{self} isn't in the curve {EC}")
                 
    def neg(self):
        return Point(self.EC, self.x, -self.y % self.p)
    
    
    
    
    def add(self,P):
        if P.x==0 and P.y==0:
           return self 
        elif self.x == P.x and self.y == -P.y % self.p :
            return Point(self.EC,0,0) 
        else:
            lam=0
            if self!= P :
               lam = ( (P.y - self.y) * inverse((P.x - self.x) ,self.p ) ) % self.p
            else : 
               lam = ((3 * (self.x**2) + self.EC.a) * inverse((2 * self.y) ,self.p)) % self.p
            x_r = (lam**2 - self.x - P.x) % self.p
            y_r =  (lam*(self.x - x_r) - self.y) % self.p
            return Point(self.EC,x_r, y_r)
    
    
    def sub(self, p):
        return self.add(neg(p))
    
    
    def mul(self, n):
        if not isinstance(n, int):
            raise Exception ("we can't multiply with a non integer")
        else:
            if n<0:
                return -self * -n
            if n == 0:
                return Point(self.curve, 0, 0) 
            #Double_and_Add
            else:
                Q = self
                R = self if n & 1 == 1 else Point(self.EC, 0, 0) 
                
                i = 2
                while i<=n:
                    Q = Q.add(Q)
                    if n & i == i:
                        R = Q.add(R)
                    i = i << 1
        return R
    
    def isEqual(self, p):
        if(self.EC == p.EC and self.x == p.x and self.y == p.y):
            return True
        else:
            return False
        
    def __str__(self):
        return (f"({self.x},{self.y})")
    

<h2> Key Exchange </h2>
Suppose that Bob and Alice want to communicate through a secure chanel using the ECDH.

In [54]:
#the curve
E = EllipticCurve(a =  1, b = 2, p = 85347454784018416893325519594186155831927915289184679356907899837050668265637)
#the generator
G = Point(E, 78914171132391128638159299996260355171298815597163094246018396104959376607073, 24295064375280632188902228344690963867143996439980967976867547372543480475204)


print( (pow(y,2,p) == ((pow(x,3,p) + x + 2)%p) ))
def getKey():
    n = randint(1,99999)
    P = G.mul(n)
    return n,P

#Alice's Key
na ,Pa= getKey()
#Alice_Private_Key = na
print(f"Alice's_Public_Key = {Pa}")
#Bob's Key
nb ,Pb= getKey()
#Bob_Private_Key = nb
print(f"Bob's_Public_Key = {Pb}")

assert  Pb.mul(na).x == Pa.mul(nb).x


shared_secret = Pb.mul(na).x 




False
Alice's_Public_Key = (29640370159964989253147211432020650536680501697227600599500389899056410553043,29086827452905640286899205154769966697052048260960522212782799720862308744862)
Bob's_Public_Key = (35689366576939404200968533102141019184504578752188814464568620536917431655871,34939484593681951475448546773680449670424933115512426277776032131475863831935)


In [57]:
#this is a sample of the encryption using AES-CBC mode 

Message=b'this is a secret message'

def encrypt(shared_secret: int):
    # Derive AES key from shared secret
    sha1 = hashlib.sha1()
    sha1.update(str(shared_secret).encode('ascii'))
    key = sha1.digest()[:16]
    iv = os.urandom(16)
    cipher = AES.new(key, AES.MODE_CBC, iv)
    ciphertext = cipher.encrypt(pad(Message, 16))
    # Prepare data to send
    data = {}
    data['iv'] = iv.hex()
    data['encrypted_message'] = ciphertext.hex()
    return data

encrypted=encrypt(shared_secret)
print(f"Alice send to Bob : {encrypted}")

def is_pkcs7_padded(message):
    padding = message[-message[-1]:]
    return all(padding[i] == len(padding) for i in range(0, len(padding)))


def decrypt(shared_secret: int, iv: str, ciphertext: str):
    # Derive AES key from shared secret
    sha1 = hashlib.sha1()
    sha1.update(str(shared_secret).encode('ascii'))
    key = sha1.digest()[:16]
    ciphertext = bytes.fromhex(ciphertext)
    iv = bytes.fromhex(iv)
    cipher = AES.new(key, AES.MODE_CBC, iv)
    plaintext = cipher.decrypt(ciphertext)

    if is_pkcs7_padded(plaintext):
        return unpad(plaintext, 16).decode('ascii')
    else:
        return plaintext.decode('ascii')


decrypted=decrypt(shared_secret,encrypted['iv'],encrypted['encrypted_message'])
print(f" the decrypted message is : {decrypted} ")

Alice send to Bob : {'iv': 'e586b779131ef53aae86473a00991051', 'encrypted_message': '66e921082554c8576616d93c5ebb03d46f89e6d48e51db3f9e0fb96a796dba9e'}
 the decrypted message is : this is a secret message 
